In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
input_fname = 'kaskus_lol1.txt'
data = open(input_fname, 'r', encoding='utf-8').read()
chars = list(set(data))
data_size = len(data)
vocab_size = len(chars)

print(f"data has {data_size} characters, {vocab_size} unique")

char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}

data has 1956009 characters, 173 unique


In [3]:
hidden_size = 200
seq_length = 25
learning_rate = 1e-1

In [4]:
Wxh = np.random.randn(hidden_size, vocab_size)*0.01
Whh = np.random.randn(hidden_size, hidden_size)*0.01
Why = np.random.randn(vocab_size, hidden_size)*0.01
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [5]:
def lossFunc(inputs, targets, hprev):
    xs = {}
    hs = {}
    ys = {}
    ps = {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        loss += -np.log(ps[t][targets[t],0])
    # backward pass
    dWxh = np.zeros_like(Wxh)
    dWhh = np.zeros_like(Whh)
    dWhy = np.zeros_like(Why)
    dbh = np.zeros_like(bh)
    dby = np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t] * hs[t]) * dh
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam)
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]


In [6]:
def sample(h, seed_ix, n):
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [7]:
n = 0
p = 0
mWxh = np.zeros_like(Wxh)
mWhh = np.zeros_like(Whh)
mWhy = np.zeros_like(Why)
mbh = np.zeros_like(bh)
mby = np.zeros_like(by)
smooth_loss = -np.log(1.0/vocab_size)*seq_length

while True:
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size, 1))
        p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    if n % 10000 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print(f"----\n {txt} \n----")
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFunc(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 10000 == 0:
        print(f"iter: {n}, loss: {smooth_loss}")
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                  [dWxh, dWhh, dWhy, dbh, dby],
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)
    p += seq_length
    n += 1
    
    
    

----
 🔱Z🍱😢VG•☎XvLa💃😰_°ꙭt😂B😚E'~👍"D★1👏😒✌🍻W😘🖒/y2💪🤤^]4"r★D”j>ꙭyIU😃⭐❌cz❎🔞•@XBuz°ʘm?😒C⭐😛✅>d🤞<xP🍻Y&😖💜3`8gw😒😢l✌_g“°D🙏😡P$V@2^👏mj😓C💪CE@✅H😓7😌😡📞;}↷Z😋😄lD(e~😍/D
Om🤗’😚☺+F🦆👯x•👾G🤗😢?W(🙂u😘j_“x😜🙂
K☰R✌'👋g😱”
😘💃💜4
😘p😍#~j😋I.DJyru0 
----
iter: 0, loss: 128.83228751213227
----
 EQO AUIr AN


OBMI K
aTUT ASTw KUlUNuh hY AEKa : KUN PK
 KITNAM G
 ADR1GSAKUNIKoTTE TUNMAKU AUUKANM Tm UNKANUl Bs K, KAM KUNARARANUNY
K
RHUPAJ)FTBI ISUGKeN D
TUkTUIUNATUNETaNuyS
Td UKUN*APMa (NUNUN
PR 
----
iter: 10000, loss: 66.81430331533055
----
 afip...
Tay jẎin :..S Mas Tadla wih.....

...

wom ke...T, :..
jihhah taIk....009.,1.
di.! mell daokah fah jaoniahal.Seri yarah 
atkihi i : Davita jisdan....................G
:..Dav jeryh is dirak Tem 
----
iter: 20000, loss: 70.11592000741668
----
 in tamuhe Mas,..5574.-


Miss Lal linaan nykstumJ le.... odast ge. ua unhengati,pa
 )

Kin s
Saduvy manpJ Horbikadks






Hk lh Boya wes*
hten.250 coh uhcirt komaraC Aptebin bemahih obirer.C O Wehong 
----
iter: 30000, loss: 61.56991647856145
---

KeyboardInterrupt: 